<a href="https://colab.research.google.com/github/liviadelgaudio/CDMO-2022-2023-exercises/blob/main/SMT/SMT_28_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver
from z3 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 14.0 MB/s eta 0:00:00


In [2]:
# esempio di istanza
n = 7
m = 3
dist = [
	[0, 3, 3, 6, 5, 8, 6, 2],
	[3, 0, 6, 3, 4, 7, 7, 3],
	[3, 4, 0, 7, 6, 3, 9, 3],
	[6, 3, 7, 0, 5, 6, 7, 4], 
	[5, 4, 6, 3, 0, 3, 3, 3],
	[6, 7, 3, 6, 3, 0, 2, 4],
	[6, 7, 5, 6, 3, 2, 0, 4],
	[2, 3, 3, 4, 3, 4, 4, 0]];
load = [15,10,7];
size = [3,2,6,8,5,4,4];


In [3]:
def display_mlp(sol, m_couriers, n_items):
    for c in range(1,m_couriers+1):
        print('Courier %i' %c)
        for n in range(n_items+2): #timesteps j
            is_working = False
            for s in range(n_items+1): #items k 
                if (s, n, c-1) in sol:
                    is_working = True
                    print('  courier %i carries item %i at time %i' % (c, s, n ))
            if not is_working:
                print('  Courier %i does not work at time %i'.format(c, n))

In [50]:

def smt_solver(n, m, dist, load, size):
  # Create all the variables, goesTo[i, j, k] is True if item i is assigned to courier k on time j.
  goesTo = [[[Bool(f"x_{i}_{j}_{k}") for k in range(m)] for j in range(n+2)] for i in range(n+1)]
  tours = [[Int('tours_{}_{}'.format(i,j)) for j in range(m)] for i in range(n+2)] 


  s = Optimize()


  #in ogni time step item viene assegnato al più un item a ciascun corriere
  for j in range(n+2):
    for k in range(m):
      s.add(Sum([goesTo[i][j][k] for i in range(n+1)])==1)

  #ciascun corriere parte e termina in n+1
  for k in range(m):
    s.add(And(goesTo[n][0][k]==True, goesTo[n][n+1][k]==True))

  #ciascun item è assegnato a un solo corriere -> 1) se un item i è assegnato al corriere k, implica che per tutti gli altri corrieri la riga i sarà 0
  #2)se un item i è assegnato al corriere k, implica che tutti gli altri elementi della riga i saranno 0
  for i in range(n):
    for j in range(1,n+1):
      for k in range(m):
        s.add(Implies(goesTo[i][j][k], Sum([goesTo[i][t][c] for t in range(1,n+1) for c in range(m) if c!=k])==0))
        s.add(Implies(goesTo[i][j][k], And([goesTo[i][t][k]==False for t in range(1,n+1) if t != j])))

  #vincolo sul load: per ciascun corriere, la somma degli item che gli sono assegnati non deve eccedere la sua load
  for k in range(m):
    for j in range(1,n+1):
      s.add(Sum([If(goesTo[i][j][k]==True, size[i], 0) for i in range(n)])<= load[k])

  #se un corriere è in posizione n+1 ad un time step diverso da 0, implica che sarà in posizione n+1 anche in tutti i timestep successivi
  for j in range(1,n+1):
    for k in range(m):
      s.add(Implies(goesTo[n][j][k], And([goesTo[n][t][k]==True for t in range(j+1, n+1)])))

  #calcolo distanze ??? 
  max_distance = Int('max_distance')
  s.add(max_distance > 0)

  #tour fatto da indici i
  for k in range(m):
    for i in range(n+1):
        s.add([Implies(goesTo[i][j][k], tours[j][k]==i) for j in range(n+2)]) #così i tour vengono calcolati correttamente
        #s.add([[Implies(And(goesTo[i][j][k], goesTo[l][j+1][k]), tours[j][k]==dist[i,l]) for l in range(n+1)] for j in range(n+1)]) #alternativa: uso tours come array in cui salvare tutte le distanze che sarebbero poi da sommare -> problema con gli indici di dist, che non vengono letti come interi
  
  #s.add(max([Sum([dist[a,b] for a,b in zip([tours[j][k] for j in range(n+1)], [tours[j][k] for j in range(1,n+2)])] for k in range(m))])==max_distance) #Uso i valori salvati in ciascun tour per calcolare distanza prendendo indici a due a due -> Non funziona perche non si accede ai valori di tour, vedi stampa sotto
  print([(a, b) for a,b in zip([tours[j][0] for j in range(n+1)], [tours[j][0] for j in range(1,n+2)])])

  
  '''
  maximum = Sum([tours[j][k] for j in range(n+2)])
  for k in range(1,m): 
    if  Sum([tours[j][k] for j in range(n+2)]) > maximum:
      maximum =  Sum([tours[j][k] for j in range(n+2)])
  s.add(max_distance == maximum)

  #optimize to minimize max distance
  
  s.minimize(max_distance)
  '''
 

  if s.check() == sat:
        model = s.model()
        for k in range(m):
          print("Tour of courier {}: {}".format(k, [model.evaluate(tours[i][k]) for i in range(n+2)]))
        #print(model.evaluate(max_distance))
        return [(i, j, k) for i in range(n+1) for j in range(n+2) for k in range(m) if model.evaluate(goesTo[i][j][k])]
        '''
        for k in range(m):
          print("Courier {}".format(k+1))
          for j in range(n+2):
            if model.evaluate(goesTo[n][j][k]): #il corriere è in posizione n+1 (origine) al time step j
              print("Courier %i does not work at time %i".format(k,j))
            else:
              print("Courier %i goes to location %i at time %i".format(k,i,j))  
          return 
          '''
  else:
        return "Unsat"  

In [49]:
display_mlp(smt_solver(n, m, dist, load, size), m, n)
#smt_solver(n, m, dist, load, size)

[(tours_0_0, tours_1_0), (tours_1_0, tours_2_0), (tours_2_0, tours_3_0), (tours_3_0, tours_4_0), (tours_4_0, tours_5_0), (tours_5_0, tours_6_0), (tours_6_0, tours_7_0), (tours_7_0, tours_8_0)]
Tour courier 0: [7, 3, 5, 7, 7, 7, 7, 7, 7]
Tour courier 1: [7, 7, 7, 7, 7, 7, 7, 7, 7]
Tour courier 2: [7, 6, 2, 4, 0, 1, 7, 7, 7]
Courier 1
  courier 1 carries item 7 at time 0
  courier 1 carries item 3 at time 1
  courier 1 carries item 5 at time 2
  courier 1 carries item 7 at time 3
  courier 1 carries item 7 at time 4
  courier 1 carries item 7 at time 5
  courier 1 carries item 7 at time 6
  courier 1 carries item 7 at time 7
  courier 1 carries item 7 at time 8
Courier 2
  courier 2 carries item 7 at time 0
  courier 2 carries item 7 at time 1
  courier 2 carries item 7 at time 2
  courier 2 carries item 7 at time 3
  courier 2 carries item 7 at time 4
  courier 2 carries item 7 at time 5
  courier 2 carries item 7 at time 6
  courier 2 carries item 7 at time 7
  courier 2 carries item 7